In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Conv2DTranspose
import matplotlib.pyplot as plt
from keras.utils import np_utils
import os 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

np.random.seed(10)
# 导入数据集，如果没有就会自动下载
(x_img_train,y_label_train),(x_img_test, y_label_test)=cifar10.load_data()
#print('train:',len(x_img_train))
#print('test :',len(x_img_test))
#print('train_image :',x_img_train.shape)
#print('train_label :',y_label_train.shape)
#print('test_image :',x_img_test.shape)
#print('test_label :',y_label_test.shape)


label_dict={0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",
            5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}

def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14) # 控制图片大小
    if num>25: num=25  #最多显示25张
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx],cmap='binary')
        title=str(i)+','+label_dict[labels[i][0]]# i-th张图片对应的类别
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[i]]
        ax.set_title(title,fontsize=10) 
        ax.set_xticks([]);
        ax.set_yticks([])        
        idx+=1 
    plt.savefig('1.png')
    plt.show()

#plot_images_labels_prediction(x_img_train,y_label_train,[],0,10)

#print(y_label_train[0])
#print(y_label_train[0][0])

#print(x_img_train[0][0][0]) #（50000，32，32，3）
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0
#print(x_img_train_normalize[0][0][0])

#print(y_label_train.shape)
#print(y_label_train[:5])

y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
#print(y_label_train_OneHot.shape)
#print(y_label_train_OneHot[:5])

np.random.seed(10)
# 载入数据集
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 
# 归一化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0
# One-Hot Encoding

y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

def create_model():
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))

    x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu', padding='same')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

    x1 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu', padding='same')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)    
    x1 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu', padding='same')(x1)
    #x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid')(x1)

    x2 = tf.keras.layers.Conv2D(filters=128,kernel_size=(3, 3),activation='relu', padding='same')(x1)
    x2 = tf.keras.layers.BatchNormalization()(x2)    
    x2 = tf.keras.layers.Conv2D(filters=128,kernel_size=(3, 3),activation='relu', padding='same')(x2)
    #x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid')(x2)

    x3 = tf.keras.layers.Conv2D(filters=256,kernel_size=(3, 3),activation='relu', padding='same')(x2)
    x3 = tf.keras.layers.BatchNormalization()(x3)    
    x3 = tf.keras.layers.Conv2D(filters=256,kernel_size=(3, 3),activation='relu', padding='same')(x3)
    #x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid')(x3)

    y = tf.keras.layers.Flatten()(x3)
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.Dense(1250, activation='relu')(y)
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.Dense(750, activation='relu')(y)
    y = tf.keras.layers.BatchNormalization()(y)

    output = tf.keras.layers.Dense(10, activation='softmax')(y)

    return tf.keras.Model(inputs=inputs, outputs=output)

model = create_model()
model.summary()
 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"  

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1)

filepath = 'cifarCnnModel.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

aug_gen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False,  # randomly flip images
)

aug_gen.fit(x_img_train_normalize)


train_history=model.fit(aug_gen.flow(x_img_train_normalize,  y_label_train_OneHot, batch_size=64), 
                        validation_data=(x_img_test_normalize, y_label_test_OneHot),
                        epochs=100,  
                        verbose=2, 
                        shuffle=True,
                        callbacks=[checkpoint, early, reduce_lr]) 

#tensorboard --logdir=C:\Users\JingZ\source\repos\PythonApplication8\PythonApplication8\logs

model = tf.keras.models.load_model('cifarCnnModel.h5')

scores = model.evaluate(x_img_test_normalize, 
                        y_label_test_OneHot, verbose=2)
print(scores[1])

rand_id = np.random.choice(range(10000), size=20)
X_pred = np.array([x_img_test_normalize[i] for i in rand_id])
y_true = [y_label_test_OneHot[i] for i in rand_id]
y_true = np.argmax(y_true, axis=1)
y_true = [label_dict[name] for name in y_true]
y_pred = model.predict(X_pred)
y_pred = np.argmax(y_pred, axis=1)
y_pred = [label_dict[name] for name in y_pred]
plt.figure(figsize=(15, 7))
for i in range(20):
    plt.subplot(4, 5, i + 1)
    plt.imshow(X_pred[i].reshape(32, 32, 3), cmap='gray')
    plt.title('True: %s \n Pred: %s' % (y_true[i], y_pred[i]), size=15)
plt.tight_layout()
plt.show()